In [15]:
from __future__ import division

import pandas as pd
import numpy as np
import sframe
import xgboost as xgb
import string
import datetime
import sframe

from nltk.corpus import stopwords # Import the stop word list

%matplotlib inline

In [16]:
train['is_duplicate'].sketch_summary()


+--------------------+----------------+----------+
|        item        |     value      | is exact |
+--------------------+----------------+----------+
|       Length       |     404290     |   Yes    |
|        Min         |      0.0       |   Yes    |
|        Max         |      1.0       |   Yes    |
|        Mean        | 0.369197853026 |   Yes    |
|        Sum         |    149263.0    |   Yes    |
|      Variance      | 0.232890798347 |   Yes    |
| Standard Deviation | 0.482587606914 |   Yes    |
|  # Missing Values  |       0        |   Yes    |
|  # unique values   |       2        |    No    |
+--------------------+----------------+----------+

Most frequent items:
+-------+--------+--------+
| value |   0    |   1    |
+-------+--------+--------+
| count | 255027 | 149263 |
+-------+--------+--------+

Quantiles: 
+-----+-----+-----+-----+-----+-----+-----+-----+------+
|  0% |  1% |  5% | 25% | 50% | 75% | 95% | 99% | 100% |
+-----+-----+-----+-----+-----+-----+-----+----

In [17]:
train.head(5)

question1,question2,is_duplicate
What is the step by stepguide to invest in share ...,What is the step by stepguide to invest in share ...,0
What is the story ofKohinoor (Koh-i-Noor) ...,What would happen if theIndian government stole ...,0
How can I increase thespeed of my internet ...,How can Internet speed beincreased by hacking ...,0
Why am I mentally verylonely? How can I solve ...,Find the remainder when[math]23^{24}[/math] is ...,0
"Which one dissolve inwater quikly sugar, s ...",Which fish would survivein salt water? ...,0


In [18]:
test = pd.read_csv('test.csv').fillna('')
test = sframe.SFrame(test)
ids = test[['test_id']]
test = test.remove_columns(['test_id'])

In [19]:
test.head(5)

question1,question2
How does the Surface Prohimself 4 compare with ...,Why did Microsoft choosecore m3 and not core i3 ...
Should I have a hairtransplant at age 24? ...,How much cost does hairtransplant require? ...
What but is the best wayto send money from China ...,What you send money toChina? ...
Which food notemulsifiers? ...,What foods fibre?
"How ""aberystwyth"" startreading? ...",How their can I startreading? ...


In [22]:
# def is_true_duplicate(x, y):
#     x = x.lower()
#     y = y.lower()
#     return x == y

def text_to_word_lists( raw_text ):
    # Convert to lower case
    words = raw_text.lower()
    # Replace punctuation with spaces
    words = words.translate(string.maketrans(string.punctuation,' '*len(string.punctuation))) 
    # Split into individual words, convert into set
    words = words.split()
    #  In Python, searching a set is much faster than searching
    #  a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # Remove stop words
    return sframe.SArray([words, [w for w in words if w not in stops]])

In [23]:
result = train['question1'].apply(lambda x : text_to_word_lists(x)).unpack()
train.add_columns(result.rename({'X.0' : 'word_list1', 'X.1' : 'word_list_no_stops1'}))

result = train['question2'].apply(lambda x : text_to_word_lists(x)).unpack()
train.add_columns(result.rename({'X.0' : 'word_list2', 'X.1' : 'word_list_no_stops2'}))

train.head(5)

Columns:
	question1	str
	question2	str
	is_duplicate	int
	word_list1	list
	word_list_no_stops1	list
	word_list2	list
	word_list_no_stops2	list

Rows: 5

Data:
+-------------------------------+-------------------------------+--------------+
|           question1           |           question2           | is_duplicate |
+-------------------------------+-------------------------------+--------------+
| What is the step by step g... | What is the step by step g... |      0       |
| What is the story of Kohin... | What would happen if the I... |      0       |
| How can I increase the spe... | How can Internet speed be ... |      0       |
| Why am I mentally very lon... | Find the remainder when [m... |      0       |
| Which one dissolve in wate... | Which fish would survive i... |      0       |
+-------------------------------+-------------------------------+--------------+
+-------------------------------+-------------------------------+
|           word_list1          |      word_list_no_stops1      |
+-------------------------------+-------------------------------+
| [what, is, the, step, by, ... | [step, step, guide, invest... |
| [what, is, the, story, of,... | [story, kohinoor, koh, noo... |
| [how, can, i, increase, th... | [increase, speed, internet... |
| [why, am, i, mentally, ver... |   [mentally, lonely, solve]   |
| [which, one, dissolve, in,... | [one, dissolve, water, qui... |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
|           word_list2          |      word_list_no_stops2      |
+-------------------------------+-------------------------------+
| [what, is, the, step, by, ... | [step, step, guide, invest... |
| [what, would, happen, if, ... | [would, happen, indian, go... |
| [how, can, internet, speed... | [internet, speed, increase... |
| [find, the, remainder, whe... | [find, remainder, math, 23... |
| [which, fish, would, survi... | [fish, would, survive, sal... |
+-------------------------------+-------------------------------+
[5 rows x 7 columns]

In [9]:
result = test['question1'].apply(lambda x : text_to_word_lists(x)).unpack()
test.add_columns(result.rename({'X.0' : 'word_list1', 'X.1' : 'word_list_no_stops1'}))

result = test['question2'].apply(lambda x : text_to_word_lists(x)).unpack()
test.add_columns(result.rename({'X.0' : 'word_list2', 'X.1' : 'word_list_no_stops2'}))

test.head(5)

Columns:
	question1	str
	question2	str
	word_list1	list
	word_list_no_stops1	list
	word_list2	list
	word_list_no_stops2	list

Rows: 5

Data:
+-------------------------------+--------------------------------+
|           question1           |           question2            |
+-------------------------------+--------------------------------+
| How does the Surface Pro h... | Why did Microsoft choose c...  |
| Should I have a hair trans... | How much cost does hair tr...  |
| What but is the best way t... | What you send money to China?  |
|  Which food not emulsifiers?  |       What foods fibre?        |
| How "aberystwyth" start re... | How their can I start reading? |
+-------------------------------+--------------------------------+
+-------------------------------+-------------------------------+
|           word_list1          |      word_list_no_stops1      |
+-------------------------------+-------------------------------+
| [how, does, the, surface, ... | [surface, pro, 4, compare,... |
| [should, i, have, a, hair,... | [hair, transplant, age, 24... |
| [what, but, is, the, best,... | [best, way, send, money, c... |
| [which, food, not, emulsif... |      [food, emulsifiers]      |
| [how, aberystwyth, start, ... | [aberystwyth, start, reading] |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
|           word_list2          |      word_list_no_stops2      |
+-------------------------------+-------------------------------+
| [why, did, microsoft, choo... | [microsoft, choose, core, ... |
| [how, much, cost, does, ha... | [much, cost, hair, transpl... |
| [what, you, send, money, t... |      [send, money, china]     |
|      [what, foods, fibre]     |         [foods, fibre]        |
| [how, their, can, i, start... |        [start, reading]       |
+-------------------------------+-------------------------------+
[5 rows x 6 columns]

In [10]:
train[['is_duplicate']].save('target', format = 'csv')
ids.save('ids', format = 'csv')

In [11]:
# train[['question1',           'question2',
#        'word_list1',          'word_list2',
#        'word_list_no_stops1', 'word_list_no_stops2']].save('train_lists.gl')

In [12]:
# test[['question1',           'question2',
#       'word_list1',          'word_list2',
#       'word_list_no_stops1', 'word_list_no_stops2']].save('test_lists.gl')